In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import xgboost
import lightgbm
from sklearn.metrics import balanced_accuracy_score as b_a_c

In [2]:
X_artificial=pd.read_csv("../data/artificial/artificial_train.data",sep=" ",header=None)
y_artificial=pd.read_csv("../data/artificial/artificial_train.labels",sep=" ",header=None)
X_digits=pd.read_csv("../data/digits/digits_train.data",sep=" ",header=None)
y_digits=pd.read_csv("../data/digits/digits_train.labels",sep=" ",header=None)

X_artificial.drop(columns=500,inplace=True)
X_digits.drop(columns=5000,inplace=True)

VT_art = VarianceThreshold(threshold = 0)
VT_art.fit(X_artificial)
X_artificial=X_artificial.iloc[:,VT_art.get_support(indices=True).tolist()]

VT_dig = VarianceThreshold(threshold = 0)
VT_dig.fit(X_digits)
X_digits=X_digits.iloc[:,VT_dig.get_support(indices=True).tolist()]
print(X_digits.shape,X_artificial.shape)



(6000, 4955) (2000, 500)


In [3]:
X_digits_train,X_digits_test,y_digits_train,y_digits_test=train_test_split(X_digits,y_digits,stratify=y_digits,test_size=0.25)

y_digits_train=y_digits_train.to_numpy().ravel()

y_digits_test=y_digits_test.to_numpy().ravel()

### no feature rescaling, no feature selection

In [4]:
logistic_model=LogisticRegression(penalty="l2",tol=1e-5,max_iter=10000)
logistic_model.fit(X_digits_train,y_digits_train)
b_a_c(y_digits_test,logistic_model.predict(X_digits_test))

0.9706666666666667

In [5]:
random_forest_model=RandomForestClassifier(n_jobs=-1) #do sprawdzenia parametry
random_forest_model.fit(X_digits_train,y_digits_train)
b_a_c(y_digits_test,random_forest_model.predict(X_digits_test))

0.974

In [6]:
xgb_model = xgboost.XGBClassifier() #do sprawdzenia parametry
xgb_model.fit(X_digits_train,(y_digits_train+1)/2)
b_a_c((y_digits_test+1)/2,xgb_model.predict(X_digits_test)),sum(xgb_model.feature_importances_>0)

(0.9793333333333334, 698)

In [7]:
adaboost_model=AdaBoostClassifier(n_estimators=100)
adaboost_model.fit(X_digits_train,y_digits_train)
b_a_c(y_digits_test,adaboost_model.predict(X_digits_test)),sum(adaboost_model.feature_importances_>0)

(0.964, 93)

In [8]:
lgm_model=lightgbm.LGBMClassifier()
lgm_model.fit(X_digits_train,y_digits_train)
b_a_c(y_digits_test,lgm_model.predict(X_digits_test)),sum(lgm_model.feature_importances_>0)

(0.9793333333333334, 993)

### feature rescaling, no feature selection

In [9]:
ss=StandardScaler()
ss.fit(X_digits_train)
X_digits_train_sc=ss.transform(X_digits_train)
X_digits_test_sc=ss.transform(X_digits_test)

In [10]:
logistic_model=LogisticRegression(penalty="l2",tol=1e-5,max_iter=1000)
logistic_model.fit(X_digits_train_sc,y_digits_train)
b_a_c(y_digits_test,logistic_model.predict(X_digits_test_sc))

0.9753333333333334

In [11]:
logistic_model=LogisticRegression(penalty="l1",tol=1e-5,max_iter=1000,solver='liblinear')
logistic_model.fit(X_digits_train_sc,y_digits_train)
b_a_c(y_digits_test,logistic_model.predict(X_digits_test_sc)),(logistic_model.coef_>0).sum()

(0.9766666666666667, 285)

In [12]:
random_forest_model=RandomForestClassifier(n_jobs=-1) #do sprawdzenia parametry
random_forest_model.fit(X_digits_train_sc,y_digits_train)
b_a_c(y_digits_test,random_forest_model.predict(X_digits_test_sc))

0.9726666666666666

In [16]:
xgb_model = xgboost.XGBClassifier() #do sprawdzenia parametry
xgb_model.fit(X_digits_train_sc,(y_digits_train+1)/2)
b_a_c((y_digits_test+1)/2,xgb_model.predict(X_digits_test_sc)),sum(xgb_model.feature_importances_>0)

(0.9793333333333334, 698)

In [14]:
adaboost_model=AdaBoostClassifier(n_estimators=100)
adaboost_model.fit(X_digits_train_sc,y_digits_train)
b_a_c(y_digits_test,adaboost_model.predict(X_digits_test_sc)),sum(adaboost_model.feature_importances_>0)

(0.964, 93)

In [15]:
lgm_model=lightgbm.LGBMClassifier()
lgm_model.fit(X_digits_train_sc,y_digits_train)
b_a_c(y_digits_test,lgm_model.predict(X_digits_test_sc)),sum(lgm_model.feature_importances_>0)

(0.98, 982)